In [1]:
import numpy as np 
import pandas as pd 
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from skimage import io

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50 
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('train.csv')
valid_df = pd.read_csv('test.csv')

In [5]:
train_df.head(10)

,img_path,class
0,mdb003.pgm.png,dense
1,mdb004.pgm.png,dense
2,mdb033.pgm.png,dense
3,mdb034.pgm.png,dense
4,mdb035.pgm.png,dense
5,mdb036.pgm.png,dense
6,mdb037.pgm.png,dense
7,mdb038.pgm.png,dense
8,mdb039.pgm.png,dense
9,mdb040.pgm.png,dense


## Setting up the image augmentation from last Lesson: 
Note that this section of the code has been pre-written for you and does not need to be changed, just run. If you would like to change the ImageDataGenerator parameters, feel free.

In [3]:
## This is the image size that VGG16 takes as input
IMG_SIZE = (224, 224)

In [6]:
train_idg = ImageDataGenerator(rescale=1. / 255.0,
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.1, 
                              width_shift_range=0.1, 
                              rotation_range=20, 
                              shear_range = 0.1,
                              zoom_range=0.1)

train_gen = train_idg.flow_from_dataframe(dataframe=train_df, 
                                         directory=None, 
                                         x_col = 'img_path',
                                         y_col = 'class',
                                         class_mode = 'binary',
                                         target_size = IMG_SIZE, 
                                         batch_size = 9
                                         )

# Note that the validation data should not be augmented! We only want to do some basic intensity rescaling here
val_idg = ImageDataGenerator(rescale=1. / 255.0
                                 )

val_gen = val_idg.flow_from_dataframe(dataframe=valid_df, 
                                         directory=None, 
                                         x_col = 'img_path',
                                         y_col = 'class',
                                         class_mode = 'binary',
                                         target_size = IMG_SIZE, 
                                         batch_size = 6) ## We've only been provided with 6 validation images

Found 20 validated image filenames belonging to 2 classes.
Found 6 validated image filenames belonging to 2 classes.


In [7]:
## Pull a single large batch of random validation data for testing after each epoch
testX, testY = val_gen.next()

## Now we'll load in VGG16 with pre-trained ImageNet weights: 

In [ ]:
model = VGG16(include_top=True, weights='imagenet')
model.summary()

In [ ]:
transfer_layer = model.get_layer('block5_pool')
vgg_model = Model(inputs=model.input,
                   outputs=transfer_layer.output)

In [ ]:
## Now, choose which layers of VGG16 we actually want to fine-tune
## Here, we'll freeze all but the last convolutional layer

## Add some code here to freeze all but the last convolutional layer:
##### Your code here ######


In [ ]:
## Check to make sure you froze the right ones: 
for layer in vgg_model.layers:
    print(layer.name, layer.trainable)

## Build a simple sequential model using only the VGG16 architecture
Note the code in the cell below has been pre-written for you, you only need to run it

In [ ]:
## Build your model using the mostly-frozen VGG16 architecture: 
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1, activation='relu'))

In [ ]:
## Set our optimizer, loss function, and learning rate (you can change the learning rate here if you'd like)
## but otherwise this cell can be run as is
optimizer = Adam(lr=1e-4)
loss = 'binary_crossentropy'
metrics = ['binary_accuracy']

In [ ]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

## Let's try another experiment where we add a few more dense layers:

In [ ]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a couple of dense (aka. fully-connected) layers.
# This is for combining features that the VGG16 model has
# recognized in the image.

##### Your code here ######


# Final output layer:
new_model.add(Dense(1, activation='relu'))

In [ ]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

## Now let's add dropout and another fully connected layer:

In [ ]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add several fully-connected layers with dropout
##### Your code here ######

# Final output layer
new_model.add(Dense(1, activation='relu'))

In [ ]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)